In [2]:
 %matplotlib inline
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from BorealWeights import BorealWeightedProblem
from gradutil import *
from pyomo.opt import SolverFactory

## Initialize data

In [3]:
revenue, carbon, deadwood, HA = init_boreal()
opt = SolverFactory('glpk')

In [4]:
carbon_clean = carbon.dropna(axis=0, how='any')
HA_clean = HA.dropna(axis=0, how='any')
deadwood_clean = deadwood.dropna(axis=0, how='any')
revenue_clean = revenue.dropna(axis=0, how='any')

# Optimization

Problem formulation

### Carbon without Nans

Lets solve the problem with just one objective and using data without Nan-values

In [5]:
min(carbon_clean)

In [6]:
%%time
bproblem = BorealWeightedProblem(carbon_clean.values)
opt.solve(bproblem.model)

In [7]:
def print_solution(problem,data):
    res_dict = dict()
    for i in problem.model.I:
        for j in problem.model.J:
            res_dict[j] = res_dict.get(j,0) + int(problem.model.x[i,j].value)
    print('Handling, # of stands')        
    for key in res_dict:
        print("{:8} {}".format(list(data)[key], res_dict[key]))

In [8]:
print_solution(bproblem, carbon_clean)

So it looks like we really are able to solve this problem using the original data! (Without Nans)

### Carbon where Nan:s as (min-1)

Lets replace Nan:s with (smallest value -1) and lets try to solve the problem. All the single optimization tasks are maximizing, so in the final solution there should be no zeros anyway. (We can check that later)

In [9]:
carbon_zeros = carbon.copy()
carbon_zeros[carbon_zeros.isnull()] = np.nanmin(carbon_zeros.values) -1

In [10]:
%%time
zero_bproblem = BorealWeightedProblem(carbon_zeros.values)
opt.solve(zero_bproblem.model)

In [11]:
res_dict = dict()
for i in zero_bproblem.model.I:
    for j in zero_bproblem.model.J:
        res_dict[j] = res_dict.get(j,0) + int(zero_bproblem.model.x[i,j].value)
print('Handling, # of stands')        
for key in res_dict:
    print("{:8} {}".format(list(carbon_clean)[key], res_dict[key]))

#### Verifying that solution looks reasonable

In [12]:
lst = values_to_list(zero_bproblem, carbon_zeros.values)

In [13]:
min(lst)

There is no zeros at all, so at least by that aspect the result looks rational.

In [14]:
max(lst)

In [15]:
np.min(carbon.dropna(axis=0, how='any').values)

In [16]:
np.max(carbon.dropna(axis=0, how='any').values)

## Solving all single objective optimization tasks

#### Solving carbon storage

In [17]:
carbon_bsolver = zero_bproblem

In [18]:
carbon_values = values_to_list(carbon_bsolver, carbon_zeros.values)
print("Minimum: {}, maximum: {}".format(min(carbon_values), max(carbon_values)))

In [19]:
np.max(carbon)

In [20]:
print_solution(carbon_bsolver, carbon_zeros)

In [21]:
this_bsolver = carbon_bsolver
sum([this_bsolver.model.x[i,j].value for i in this_bsolver.model.I for j in this_bsolver.model.J])

In [22]:
sum(carbon_values)

#### Solving HA

In [23]:
HA_zeros = HA.copy()
HA_zeros[HA_zeros.isnull()] = np.nanmin(HA_zeros.values) - 1

In [24]:
%%time
HA_bsolver = BorealWeightedProblem(HA_zeros.values)
opt.solve(HA_bsolver.model)

In [25]:
HA_values = values_to_list(HA_bsolver, HA_zeros.values)
print("Minimum: {}, maximum: {}".format(min(HA_values), max(HA_values)))

In [26]:
print_solution(HA_bsolver, HA)

In [27]:
sum(HA_values)

#### Solving deadwood

In [28]:
deadwood_zeros = deadwood.copy()
deadwood_zeros[deadwood_zeros.isnull()] = np.nanmin(deadwood_zeros.values) -1

In [29]:
%%time
deadwood_bsolver = BorealWeightedProblem(deadwood_zeros.values)
opt.solve(deadwood_bsolver.model)

In [30]:
deadwood_values = values_to_list(deadwood_bsolver, deadwood_zeros.values)
print("Minimum: {}, maximum: {}".format(min(deadwood_values), max(deadwood_values)))

In [31]:
print_solution(deadwood_bsolver, deadwood_zeros)

In [32]:
sum(deadwood_values)

#### Solving Timber revenue

In [33]:
revenue_zeros = revenue.copy()
revenue_zeros[revenue_zeros.isnull()] = np.nanmin(revenue_zeros.values) -1

In [34]:
%%time
revenue_bsolver = BorealWeightedProblem(revenue_zeros.values)
opt.solve(revenue_bsolver.model)

In [35]:
revenue_values = values_to_list(revenue_bsolver, revenue_zeros.values)
print("Minimum: {}, maximum: {}".format(min(revenue_values), max(revenue_values)))

In [36]:
print_solution(revenue_bsolver, revenue_zeros)

There were some stands with value 0.0, so lets check them:

In [37]:
rv = np.array(revenue_values)
revenue.iloc[rv == 0]

Apparently on some stands there is no way to make any profit, so it is ok that those are only zeros.

In [38]:
sum(revenue_values)

## Comparing optimizations to ones in the papers

The single objective results are documented in http://onlinelibrary.wiley.com/doi/10.1111/1365-2664.12790/full
so it is meaningful to compare our results to that one.

Results in the paper:
"The maximum capacity of the landscape 
- (i) to provide harvest revenues (NPV) was 250 M€ (average 5800 € ha−1),
- (ii) to store carbon was 4459 × 103 MgC (average 103 MgC ha−1), 
- (iii) for deadwood index was 218 150 m3 (average 5·1 m3 ha−1) and 
- (iv) for the combined habitat availability was 20 211 (no units) (average 0·47 ha−1)."

For us the correspondig values are:

In [39]:
print("(i) Harvest revenues {:.0f} M€".format(sum(revenue_values)/1000000))
print("(ii) Carbon storage {:.0f} x 100 MgC".format(sum(carbon_values)/100))
print("(iii) Deadwood index {:.0f} m3".format(sum(deadwood_values)))
print("(iv) Combined Habitat {:.0f}".format(sum(HA_values)))

Assuming the " x 103 MgC" being just type and actually meaning "x 100 MgC", there are still differences in stored carbon values and combined habitat availability values.

There is still something weird with the data values given in the paper:


In [40]:
print('Total ha-1 calculated according to the values given in paper:')
print('-'*62)
print('Revenue/(average timber revenue/ha-1) = {}'.format(250000000/5800))
print('Carbon/(average carbon storage /ha-1) = {}'.format(4459*1000/103))
print('Deadwood/(average deadwood index/ha-1) = {}'.format(218150/5.1))
print('Combined habitat/(average habitat/ha-1) = {}'.format(20211/0.47))

Now all these values indicate that there should be ~43000 hectars in total. Paper still states that there were 68 700 hectars. I don't really know if that is a real problem regarding the optimization task, but it's still odd.

Lets draw the same pictures than in the paper, so we can see if there are some deviations there also.

In [41]:
%pylab inline
pylab.rcParams['figure.figsize'] = (15,12)

ind = list(revenue)
ind.append('Öptim.')

fig, ax = plt.subplots(2,2)
val = revenue.sum().values
val = np.append(val, sum(revenue_values))
ax[0,0].bar(ind, val)
ax[0,0].set_title('Timber Harvest Revenues')

val = carbon.sum().values
val = np.append(val, sum(carbon_values))
ax[0,1].bar(ind, val)
ax[0,1].set_title('Carbon storage')

val = deadwood.sum().values
val = np.append(val, sum(deadwood_values))
ax[1,0].bar(ind, val)
ax[1,0].set_title('Deadwood')

val = HA.sum().values
val = np.append(val, sum(HA_values))
ax[1,1].bar(ind, val)
ax[1,1].set_title('Habitat availability')



These really doesn't look like the same as in the paper. Something must be wrong with the way I am summing values without optimization.

In [42]:
from IPython.display import Image
Image('../paperOptims.png')

### Pay-off Table

In [46]:
rr = sum(values_to_list(revenue_bsolver, revenue_zeros.values))
rc = sum(values_to_list(revenue_bsolver, carbon_zeros.values))
rd = sum(values_to_list(revenue_bsolver, deadwood_zeros.values))
rh = sum(values_to_list(revenue_bsolver, HA_zeros.values))

cr = sum(values_to_list(carbon_bsolver, revenue_zeros.values))
cc = sum(values_to_list(carbon_bsolver, carbon_zeros.values))
cd = sum(values_to_list(carbon_bsolver, deadwood_zeros.values))
ch = sum(values_to_list(carbon_bsolver, HA_zeros.values))

dr = sum(values_to_list(deadwood_bsolver, revenue_zeros.values))
dc = sum(values_to_list(deadwood_bsolver, carbon_zeros.values))
dd = sum(values_to_list(deadwood_bsolver, deadwood_zeros.values))
dh = sum(values_to_list(deadwood_bsolver, HA_zeros.values))

hr = sum(values_to_list(HA_bsolver, revenue_zeros.values))
hc = sum(values_to_list(HA_bsolver, carbon_zeros.values))
hd = sum(values_to_list(HA_bsolver, deadwood_zeros.values))
hh = sum(values_to_list(HA_bsolver, HA_zeros.values))


In [111]:
print('Ideal by| Revenue    Carbon   Deadwood   HA')
print('        ','-'*36)
print('Revenue  {:9.0f} | {:7.0f} | {:6.0f} | {:5.0f}'.format(rr,rc,rd,rh))
print('Carbon   {:9.0f} | {:7.0f} | {:6.0f} | {:5.0f}'.format(cr,cc,cd,ch))
print('Deadwood {:9.0f} | {:7.0f} | {:6.0f} | {:5.0f}'.format(dr,dc,dd,hh))
print('HA       {:9.0f} | {:7.0f} | {:6.0f} | {:5.0f}'.format(hr,hc,hd,hh))

In [112]:
z_ideal = np.array([rr,cc,dd,hh])
z_nadir = np.array([cr,rc,rd,rh])